In [72]:
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno
import numpy  as np
import seaborn as sns
%matplotlib inline

path = './data/'
pd.options.display.max_rows = 150
pd.options.display.max_columns = 350
plt.rc('font',family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

## 데이터로드

In [73]:
sigungu = pd.read_csv(path+'sigungu_imp_10.csv', encoding='cp949')
sigungu_3 = pd.read_csv(path+'시군구별지역안전지표_2015.csv', encoding='cp949')

In [74]:
y_bin = sigungu.iloc[:,-7:]    #등급(binary)
y_deci = sigungu_3.iloc[:,-7:] #등급(1-5)
X = sigungu.iloc[:,4:-7]       #독립변수X
name = sigungu.iloc[:,:4]      #이름

In [75]:
y_bin.columns = ['fire','transport','disaster','crime','accident','suicide','infection']

In [76]:
sigungu_xy = pd.concat([X,y_bin], axis=1)
sigungu_nxy = pd.concat([name, sigungu_xy,y_deci], axis=1)
sigungu_nxy.shape

(226, 162)

## 추가자료

In [77]:
#성비 추가
성비_2015 = pd.read_csv(path+'성비_2015.csv', encoding='cp949')
성비_2015.drop(['연령별'],axis=1, inplace=True)
성비_2015.columns=['시도','시군구','성비']
sigungu_nxy= sigungu_nxy.merge(성비_2015)
성비_2015.shape

(281, 3)

In [78]:
#외국인비율추가
transpath = 'C:/Users/COM/Desktop/교통사고/'
주민등록인구=pd.read_csv(transpath+'주민등록인구.csv', encoding='cp949')
주민등록인구.rename(columns={'전국':'시도','소계':'시군구','주민등록인구':'전체인구'}, inplace=True)
주민등록인구['외국인비율'] = 주민등록인구.등록외국인수 / 주민등록인구.전체인구*100
sigungu_nxy
sigungu_nxy=sigungu_nxy.merge(주민등록인구, on=['시도','시군구'])
주민등록인구.shape

(245, 8)

In [79]:
#제조업비율추가
manufact = pd.read_csv(transpath +'2017_제조업체_사업체.csv', encoding='cp949')
manufact.head(1)
# ex_manufact = manufact.loc[manufact['산업분류별']=='전체',['지역별(시/군/구)','2015']]
# ex_manufact.columns = ['시군구','전체산업체수']
# ex_manufact.reset_index(drop=True, inplace=True)
# manufact = manufact.loc[~(manufact['산업분류별']=='전체'),['지역별(시/군/구)','2015']]
# manufact.columns = ['시군구','제조업체수']
# manufact.reset_index(drop=True, inplace=True)
manufact['제조업비율'] = (manufact['제조사업체수']/manufact['사업체수'])
manufact.head(1)
sigungu_nxy.head(1)
sigungu_nxy = sigungu_nxy.merge(manufact, on=['시도','시군구'])
sigungu_nxy.shape
sigungu_nxy.head(1)

,년도,시도,시군구,지역,1인가구수,5대범죄 발생건수,가스사고발생건수,가해(타살) 사망자수,감염병 발생건수,감염병 사망자수,건강보험급여실적,건설업 업체수,건설업 종사자수,건축용지 면적,경찰관서수,고령인구수,공무원수(정원),공업지역 면적,교통문화지수,교통사고 발생건수,교통사고 부상자수,교통사고 사망자수,교통사고사망자수(고속도로),구거면적,구거면적 비율,구급발생건수,구조구급 발생건수,구조구급대원수,구조발생건수,급경사지 붕괴위험지구 수,기초수급자수,기초수급자수(65세이상),노인 교통사고 부상자수,노인 교통사고 사망자수,도로면적,도로면적 비율,도로연장,도시지역 면적,도시지역 면적 비율,독거노인수,무면허교통사고 발생건수,반지하가구 수,범죄발생건수(강간.강제추행),범죄발생건수(강도),범죄발생건수(도박),범죄발생건수(마약),범죄발생건수(방화),범죄발생건수(살인),범죄발생건수(약취.유인),범죄발생건수(절도),범죄발생건수(폭력),병상수,보건업 및 사회복지서비스업 종사자수,보행사상자수,보행자전용·우선도로 면적,보행자전용·우선도로 연장,비닐하우스 면적,빈집수,산사태위험지역 면적,상업지역 면적,수계밀도,순이동자수,스트레스인지율,시가화율,시군구내 전입자수,시군구외 전입자수,시도간 전입자수,시도내-시군구간 전입자수,실업률,야간인구수,양호한 주관적 건강수준 인지율,어린이 교통사고 부상자수,어린이 교통사고 사망자수,어린이 아토피,어린이 천식,외국인수,우울감경험률,운전시 안전벨트 착용률,월간음주율,유지면적,유지면적 비율,유치원생수,음식점 및 주점업 업체수,음식점 및 주점업 종사자수,음주교통사고 발생건수,의료기관수(요양기관),의료보장 공.교 가입자수,의료보장 근로자 사업장수,의료보장 사업장수,의료보험료,의료인력,이혼건수,익사자수,인구밀도,인플루엔자 예방접종률,임야 면적,자동차등록대수,자살 사망자수,자연재해 발생건수(10년평균),자연재해 사망자수,자연재해 사망자수(10년평균),자연재해 피해액(10년평균),자연재해 피해자수(10년평균),자연재해위험개선지구 수,장애인수,재난약자수,재정자립도,재정자주도,전기화재발생건수,제방면적,제방면적 비율,제조업 업체수,제조업 종사자수,주간인구지수,주거지역 면적,주민등록인구,주민등록인구(14세이하),주민등록인구(60세이상),주민등록인구(여자),주민등록인구(청소년),지역안전도(관리능력),지역안전도(등급),지역안전도(방재성능),지역안전도(위험환경),지역안전도(점수),질병이환 및 사망외인으로 인한 사망자수,창고 및 운송관련 서비스업 업체수,초등학생수,총 사업체수,총전입자수,추락 사망자수,특수의료장비수,하천면적,하천면적 비율,하천연장,학교수(고등학교),학교수(중학교),학교수(초등학교),해안선 길이,행정구역 면적,혼인귀화자수,화재 발생건수,화재 사망자수,화재 피해액,화재 피해자수,화재구조실적,화재사망자(환산),화재피해 경감액,fire,transport,disaster,crime,accident,suicide,infection,화재,교통,자연재해,범죄,안전사고,자살,감염병,성비,천명당외국인수,등록외국인수,전체인구,남,녀,외국인비율,제조사업체수,사업체수,제조업비율
0,2015,강원도,강릉시,2015_강원도_강릉시,1161.45445,121.96083,0.04634,0.13901,25.67109,2.03886,1559476.199,37.53354,335.85565,0.02344,0.3707,1668.57424,58.84888,0.00457,77.36,228.30585,392.99003,0.83408,0.04634,0.00641,0.64111,32.34371,44.85489,2.22421,12.51118,0.41704,314.30862,108.15219,36.42143,0.32436,0.02316,2.31551,660.14293,0.13173,13.17299,404.29643,53.98342,22.52012,4.63377,0.50971,1.29746,0.55605,0.50971,0.13901,0.0,48.65458,68.02374,108.61557,309.39682,19.69352,0.0,0.0,0.0817,229.04725,99.99013,0.00176,4.9,-37.53354,23.2,2.25,773.56156,598.26604,406.61332,191.65273,2.0,9931.4202,47.9,27.61727,0.0,98.65296,89.98781,76.78157,4.7,73.9,59.8,0.00471,0.47098,70.20162,201.29097,489.5578,53.98342,14.78173,458.8359,222.69899,224.83052,6229719.75,0.41704,21.91773,0.09268,209.07,32.2,0.80524,4428.91102,3.66068,0.00231,0.0,0.0,5886.61005,0.0,0.04634,605.49472,2261.65046,17.39,64.61,2.27055,0.00123,0.12303,53.56638,300.31463,99.4,0.01616,215807.0,1283.55429,2266.60859,5034.96179,1200.47079,0.741,4.0,0.438,0.553,59.98,7.08967,4.68011,522.87461,2564.56,1371.82761,0.78774,1.52914,0.02068,2.06752,0.02235,0.50971,0.55605,1.62182,0.07086,1040.37773,9.91627,10.79668,0.09268,72985.02829,0.88042,1.57548,0.17458,271425.0696,1,0,1,1,1,1,1,4,3,4,4,4,4,4,101.4,8.25,1770,214560,106545,108015,0.824944,1263,20518,0.061556


In [80]:
sigungu_nxy.shape

(225, 172)

In [81]:
# for i in range(len(sigungu_nxy)):
#     print(sigungu_nxy.corr().reset_index().loc[:,['index','제조업업체수']][i])
# # sigungu_nxy.head(3)
# # sigungu_nxy.corr().reset_index()
# for i,var in var(sigungu_nxy):
#     print(sigungu_nxy.corr().reset_index().loc[:,['index','제조업업체수']][i])

# -표준화
<pre>
표준화하지 않고 그래프를 그립니다.
그래프를 직관적으로 해석하기 위함입니다.
예로,
이혼건수 : (scale된 지표) 0.5보다는
이혼건수 : (만명당) 20건이 더 직관적입니다.
</pre>

# -함수선언
### 중앙선그리는 함수
scatter(x="이혼건수", y="5대범죄 발생건수", color="crime", df=sigungu_nxy, medianline=1, text=1, palette='picnic', hovername='지역')

In [82]:
import plotly.express as px
import plotly.graph_objects as go
def scatter(x, y, color, df, hovername=None,text=False, medianline=False, palette='fall', trendline=None):
    if text == True:
        fig = px.scatter(df, x=x, y=y, text='시군구',trendline=trendline,
                         hover_name=hovername, color=color, color_continuous_scale=palette)
        fig.update_traces(textposition='top center')
    else:
        fig = px.scatter(df, x=x, y=y,trendline=trendline,
                         hover_name=hovername, color=color, color_continuous_scale=palette)
        fig.update_traces(textposition='top center')
    if medianline==True:
        #x축 중앙값 그리기
        fig.add_trace(go.Scatter(x=[df[x].median()]*2,
                       y=[-10000,10000],
                       name="median x: "+str(round(df[x].median(),2)),
                       mode="lines",
                       line = dict(color = ('rgb(0,0,0)'),width=0.5),
                       visible=True,))
        #y축 중앙값 그리기
        fig.add_trace(go.Scatter(x=[-10000,10000],
                       y=[df[y].median()]*2,
                       name="median y: "+str(round(df[y].median(),2)),
                       mode="lines",
                       line = dict(color = ('rgb(0,0,0)'),width=0.5),
                       visible=True))
        fig.update_layout(legend=dict(x=-.05, y=1.15))
    #플롯 레이아웃(x범위, y범위) 설정
    fig.update_xaxes(range=[df[x].min()*0.8, df[x].max()*1.05]) #min값 음수인지 아닌지 확인 후 범위 조절
    fig.update_yaxes(range=[df[y].min()*0.5, df[y].max()*1.05]) #min값 음수인지 아닌지 확인 후 범위 조절
    fig.show()

### 회귀식기준 위아래 갯수세기 함수
total_regression(x_='교통사고 발생건수',y_='음주교통사고 발생비율', totaldf=sigungu_nxy,groupdf=group2)

In [83]:
from scipy import stats
def total_regression(x_,y_, totaldf,groupdf):
    #전체에 대한 xy
    x = totaldf[x_].tolist()
    y = totaldf[y_].tolist()
    #전체에 대한 회귀식생성
    slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
    #지표출력
    print(stats.linregress(x,y))
    print('Rsquared=',stats.linregress(x,y)[2]**2)
        #group에 포함되는 xy
    x = groupdf[x_].tolist()
    y = groupdf[y_].tolist()
    y_pred = []
    for i in range(len(x)):
        y_pred.append(slope*np.array(x[i])+intercept)
    upper = 0
    lower = 0
    for i in range(len(y)):
        if y[i]>=y_pred[i]:
            upper +=1
        else:
            lower +=1
    return pd.DataFrame({'':['total','upper','lower'],
                          'count':[len(y),upper,lower],
                          'ratio':[len(y)/len(y), upper/len(y), lower/len(y)]})

# -가설설정
### 제조업체수와 종사자수가 많고 교통등급이 낮은지역에서는 음주교통사고비율이 높을 것이다.
<pre>
(기각)제조업체수와 종사자수가 많은 공단지역에서는 음주단속비율이 높아 오히려 음주사망사고가 낮을 것이다.
(기각)제조업체수와 종사자수가 많은 공장단지에서는 음주교통사고가 많을 것이다.</pre>

# -가설검증


### 1. 가설검증의 대상이 되는 집단을 그룹화

In [84]:
#1)두개의 변수와 중앙값을 기준으로 각 사분면이 분리된다.
scatter(x="제조업 업체수", y="제조업 종사자수", color="transport", df=sigungu_nxy,
        medianline=True, hovername='지역', palette='fall')

In [85]:
#2)데이터프레임의 새로운 열을 만들어 타겟팅 그룹을 구분한다.
#1사분면의 모든 주황초록점 : group1, 1사분면 주황점: group2
sigungu_nxy['group1'] = np.zeros(len(sigungu_nxy))
sigungu_nxy['group2'] = np.zeros(len(sigungu_nxy))
#조건을 만족하는 행 인덱스 가져오기
cond = (sigungu_nxy['제조업 업체수']>sigungu_nxy['제조업 업체수'].median())&((sigungu_nxy['제조업 종사자수']>sigungu_nxy['제조업 종사자수'].median()))
group1_index = sigungu_nxy.loc[cond].index
group2_index = sigungu_nxy.loc[cond&sigungu_nxy.transport==1].index
#group1에 속하는 애들을 1로 표시
sigungu_nxy.loc[sigungu_nxy.index.isin(group1_index),'group1'] = 1
sigungu_nxy.loc[sigungu_nxy.index.isin(group2_index),'group2'] = 1

### 2. 가설검증에 필요한 지표를 새로운 열로 추가
외부데이터수집, 데이터변형, 데이터프레임화

In [86]:
#음주사고비율 추가(음주교통사고/일반교통사고)
sigungu_nxy['음주교통사고 발생비율'] = sigungu_nxy['음주교통사고 발생건수']/sigungu_nxy['교통사고 발생건수']

In [87]:
#추가된 열(음주교통사고 발생비율) 확인
sigungu_nxy.head(1)

,년도,시도,시군구,지역,1인가구수,5대범죄 발생건수,가스사고발생건수,가해(타살) 사망자수,감염병 발생건수,감염병 사망자수,건강보험급여실적,건설업 업체수,건설업 종사자수,건축용지 면적,경찰관서수,고령인구수,공무원수(정원),공업지역 면적,교통문화지수,교통사고 발생건수,교통사고 부상자수,교통사고 사망자수,교통사고사망자수(고속도로),구거면적,구거면적 비율,구급발생건수,구조구급 발생건수,구조구급대원수,구조발생건수,급경사지 붕괴위험지구 수,기초수급자수,기초수급자수(65세이상),노인 교통사고 부상자수,노인 교통사고 사망자수,도로면적,도로면적 비율,도로연장,도시지역 면적,도시지역 면적 비율,독거노인수,무면허교통사고 발생건수,반지하가구 수,범죄발생건수(강간.강제추행),범죄발생건수(강도),범죄발생건수(도박),범죄발생건수(마약),범죄발생건수(방화),범죄발생건수(살인),범죄발생건수(약취.유인),범죄발생건수(절도),범죄발생건수(폭력),병상수,보건업 및 사회복지서비스업 종사자수,보행사상자수,보행자전용·우선도로 면적,보행자전용·우선도로 연장,비닐하우스 면적,빈집수,산사태위험지역 면적,상업지역 면적,수계밀도,순이동자수,스트레스인지율,시가화율,시군구내 전입자수,시군구외 전입자수,시도간 전입자수,시도내-시군구간 전입자수,실업률,야간인구수,양호한 주관적 건강수준 인지율,어린이 교통사고 부상자수,어린이 교통사고 사망자수,어린이 아토피,어린이 천식,외국인수,우울감경험률,운전시 안전벨트 착용률,월간음주율,유지면적,유지면적 비율,유치원생수,음식점 및 주점업 업체수,음식점 및 주점업 종사자수,음주교통사고 발생건수,의료기관수(요양기관),의료보장 공.교 가입자수,의료보장 근로자 사업장수,의료보장 사업장수,의료보험료,의료인력,이혼건수,익사자수,인구밀도,인플루엔자 예방접종률,임야 면적,자동차등록대수,자살 사망자수,자연재해 발생건수(10년평균),자연재해 사망자수,자연재해 사망자수(10년평균),자연재해 피해액(10년평균),자연재해 피해자수(10년평균),자연재해위험개선지구 수,장애인수,재난약자수,재정자립도,재정자주도,전기화재발생건수,제방면적,제방면적 비율,제조업 업체수,제조업 종사자수,주간인구지수,주거지역 면적,주민등록인구,주민등록인구(14세이하),주민등록인구(60세이상),주민등록인구(여자),주민등록인구(청소년),지역안전도(관리능력),지역안전도(등급),지역안전도(방재성능),지역안전도(위험환경),지역안전도(점수),질병이환 및 사망외인으로 인한 사망자수,창고 및 운송관련 서비스업 업체수,초등학생수,총 사업체수,총전입자수,추락 사망자수,특수의료장비수,하천면적,하천면적 비율,하천연장,학교수(고등학교),학교수(중학교),학교수(초등학교),해안선 길이,행정구역 면적,혼인귀화자수,화재 발생건수,화재 사망자수,화재 피해액,화재 피해자수,화재구조실적,화재사망자(환산),화재피해 경감액,fire,transport,disaster,crime,accident,suicide,infection,화재,교통,자연재해,범죄,안전사고,자살,감염병,성비,천명당외국인수,등록외국인수,전체인구,남,녀,외국인비율,제조사업체수,사업체수,제조업비율,group1,group2,음주교통사고 발생비율
0,2015,강원도,강릉시,2015_강원도_강릉시,1161.45445,121.96083,0.04634,0.13901,25.67109,2.03886,1559476.199,37.53354,335.85565,0.02344,0.3707,1668.57424,58.84888,0.00457,77.36,228.30585,392.99003,0.83408,0.04634,0.00641,0.64111,32.34371,44.85489,2.22421,12.51118,0.41704,314.30862,108.15219,36.42143,0.32436,0.02316,2.31551,660.14293,0.13173,13.17299,404.29643,53.98342,22.52012,4.63377,0.50971,1.29746,0.55605,0.50971,0.13901,0.0,48.65458,68.02374,108.61557,309.39682,19.69352,0.0,0.0,0.0817,229.04725,99.99013,0.00176,4.9,-37.53354,23.2,2.25,773.56156,598.26604,406.61332,191.65273,2.0,9931.4202,47.9,27.61727,0.0,98.65296,89.98781,76.78157,4.7,73.9,59.8,0.00471,0.47098,70.20162,201.29097,489.5578,53.98342,14.78173,458.8359,222.69899,224.83052,6229719.75,0.41704,21.91773,0.09268,209.07,32.2,0.80524,4428.91102,3.66068,0.00231,0.0,0.0,5886.61005,0.0,0.04634,605.49472,2261.65046,17.39,64.61,2.27055,0.00123,0.12303,53.56638,300.31463,99.4,0.01616,215807.0,1283.55429,2266.60859,5034.96179,1200.47079,0.741,4.0,0.438,0.553,59.98,7.08967,4.68011,522.87461,2564.56,1371.82761,0.78774,1.52914,0.02068,2.06752,0.02235,0.50971,0.55605,1.62182,0.07086,1040.37773,9.91627,10.79668,0.09268,72985.02829,0.88042,1.57548,0.17458,271425.0696,1,0,1,1,1,1,1,4,3,4,4,4,4,4,101.4,8.25,1770,214560,106545,108015,0.824944,1263,20518,0.061556,0.0,0.0,0.236452


### 3. 산점도를 이용한 가설검증

In [89]:
#1)산점도로 회귀선 기준 타겟팅지역의 분포를 확인
# scatter(x="교통사고 발생건수", y="음주교통사고 발생비율", color="group2", df=sigungu_nxy,
#         medianline=1, palette='picnic', hovername='지역', trendline='ols')
scatter(x="교통사고 발생건수", y="음주교통사고 발생건수", color="group2", df=sigungu_nxy,
        medianline=1, palette='picnic', hovername='지역', trendline='ols')

In [91]:
#2)회귀식기준 위아래 비율 비교
group2 = sigungu_nxy.loc[sigungu_nxy.group2==1]
total_regression(x_='교통사고 발생건수',y_='음주교통사고 발생건수', totaldf=sigungu_nxy,groupdf=group2)

LinregressResult(slope=0.1338744307398634, intercept=19.903327511470827, rvalue=0.6445062175128073, pvalue=8.339258322897315e-28, stderr=0.010635351767405677)
Rsquared= 0.41538826441266613


,,count,ratio
0,total,40,1.0
1,upper,28,0.7
2,lower,12,0.3


In [63]:
#우리가 생각했던 매개변수들이
#음주운전이라는 것을 얼마나 잘 설명할 수 있는가?(유의한식으로)
import statsmodels.api as sm
group2.rename(columns={'음주교통사고 발생비율':'음주교통사고발생비율',
                       '음주교통사고 발생건수':'음주교통사고발생건수',
                       '주민등록인구(여자)':'주민등록인구여자',
              '제조업 업체수':'제조업업체수',
              '제조업 종사자수':'제조업종사자수',
              '교통사고 발생건수':'교통사고발생건수'}, inplace=True)
reg = sm.OLS.from_formula("음주교통사고발생비율 ~ 제조업비율+성비+외국인비율", group2).fit()
reg = sm.OLS.from_formula("음주교통사고발생건수 ~ 제조업업체수+주민등록인구여자+외국인수", group2).fit()
reg.summary()
# 제조업업체수+제조업종사자수+교통사고발생건수: P0.04, R0.19
# 제조업업체수+제조업종사자수: P0.07(X), R0.13
# 제조업업체수: P0.03, R0.11

C:\Anaconda3\lib\site-packages\pandas\core\frame.py:4223: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             음주교통사고발생건수   R-squared:                       0.329
Model:                            OLS   Adj. R-squared:                  0.273
Method:                 Least Squares   F-statistic:                     5.874
Date:                Wed, 04 Dec 2019   Prob (F-statistic):            0.00226
Time:                        14:53:19   Log-Likelihood:                -174.86
No. Observations:                  40   AIC:                             357.7
Df Residuals:                      36   BIC:                             364.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   -445.5705    192.752     -2.312      0.027    -836.490     -54.651
제조업업체수         0.0613      0.028      2.221      0.033       0.005       0.117
주민등록인구여자       0.0971      0.038      2.540      0.016       0.020       0.175
외국인수           0.0344      0.024      1.452      0.155      -0.014       0.083
==============================================================================
Omnibus:                       35.937   Durbin-Watson:                   2.025
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              120.741
Skew:                           2.092   Prob(JB):                     6.04e-27
Kurtosis:                      10.412   Cond. No.                     3.01e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.01e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [64]:
#우리가 생각했던 매개변수들이
#음주운전이라는 것을 얼마나 잘 설명할 수 있는가?(유의한식으로)
import statsmodels.api as sm
sigungu_nxy.rename(columns={'음주교통사고 발생비율':'음주교통사고발생비율',
                      '제조업 업체수':'제조업업체수',
                      '제조업 종사자수':'제조업종사자수',
                      '교통사고 발생건수':'교통사고발생건수',
                      '교통사고 사망자수':'교통사고사망자수',
                      '교통사고 부상자수':'교통사고부상자수',
                      '의료보장 사업장수':'의료보장사업장수',
                            '음주교통사고 발생건수':'음주교통사고발생건수',
                            '주민등록인구(여자)':'주민등록인구여자',
                      '1인가구수':'일인가구수',
                      '학교수(중학교)':'중학교수',
                      '음식점 및 주점업 종사자수':'음식점및주점업종사자수',
                      '음식점 및 주점업 업체수':'음식점및주점업업체수',
                      '지역안전도(점수)':'지역안전도'}, inplace=True)
reg = sm.OLS.from_formula("음주교통사고발생비율 ~ 제조업업체수+성비+외국인수", sigungu_nxy).fit()
reg = sm.OLS.from_formula("음주교통사고발생비율 ~ 제조업비율+성비+외국인비율", sigungu_nxy).fit()
reg = sm.OLS.from_formula("음주교통사고발생건수 ~ 제조업업체수+주민등록인구여자+외국인수", sigungu_nxy).fit()
reg.summary()
# 제조업업체수+제조업종사자수+transport: P0.000, R0.08
# 전체에 대해서

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             음주교통사고발생건수   R-squared:                       0.207
Model:                            OLS   Adj. R-squared:                  0.196
Method:                 Least Squares   F-statistic:                     19.23
Date:                Wed, 04 Dec 2019   Prob (F-statistic):           4.06e-11
Time:                        14:53:19   Log-Likelihood:                -905.26
No. Observations:                 225   AIC:                             1819.
Df Residuals:                     221   BIC:                             1832.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -87.2742     38.859     -2.246      0.026    -163.856     -10.692
제조업업체수         0.0724      0.013      5.666      0.000       0.047       0.098
주민등록인구여자       0.0253      0.008      3.276      0.001       0.010       0.040
외국인수           0.0069      0.007      1.039      0.300      -0.006       0.020
==============================================================================
Omnibus:                      113.389   Durbin-Watson:                   1.699
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              985.456
Skew:                           1.753   Prob(JB):                    1.03e-214
Kurtosis:                      12.634   Cond. No.                     2.13e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.13e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [65]:
group1 = sigungu_nxy.loc[sigungu_nxy.group1==1]

In [66]:
group1.rename(columns={'음주교통사고 발생비율':'음주교통사고발생비율',
                      '제조업 업체수':'제조업업체수',
                      '제조업 종사자수':'제조업종사자수',
                      '교통사고 발생건수':'교통사고발생건수',
                      '교통사고 사망자수':'교통사고사망자수',
                      '교통사고 부상자수':'교통사고부상자수',
                      '의료보장 사업장수':'의료보장사업장수',
                      '1인가구수':'일인가구수',
                      '학교수(중학교)':'중학교수',
                      '음식점 및 주점업 종사자수':'음식점및주점업종사자수',
                      '음식점 및 주점업 업체수':'음식점및주점업업체수',
                       '음주교통사고 발생건수':'음주교통사고발생건수',
                            '주민등록인구(여자)':'주민등록인구여자',
                      '지역안전도(점수)':'지역안전도'}, inplace=True)
reg = sm.OLS.from_formula("음주교통사고발생비율 ~ 제조업업체수+성비+외국인수", group1).fit()
reg = sm.OLS.from_formula("음주교통사고발생비율 ~ 제조업비율+성비+외국인비율", group1).fit()
reg = sm.OLS.from_formula("음주교통사고발생건수 ~ 제조업업체수+주민등록인구여자+외국인수", group1).fit()
reg.summary()
# 제조업업체수+제조업종사자수+transport: P0.002, R0.163
# group1에 대해서
# 교통사고 사망자수
# 교통사고 부상자수
#+독거노인수+교통사고사망자수+일인가구수++중학교수

C:\Anaconda3\lib\site-packages\pandas\core\frame.py:4223: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             음주교통사고발생건수   R-squared:                       0.257
Model:                            OLS   Adj. R-squared:                  0.230
Method:                 Least Squares   F-statistic:                     9.248
Date:                Wed, 04 Dec 2019   Prob (F-statistic):           2.54e-05
Time:                        14:53:20   Log-Likelihood:                -350.23
No. Observations:                  84   AIC:                             708.5
Df Residuals:                      80   BIC:                             718.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   -349.3191    120.848     -2.891      0.005    -589.815    -108.824
제조업업체수         0.0612      0.018      3.325      0.001       0.025       0.098
주민등록인구여자       0.0777      0.024      3.233      0.002       0.030       0.126
외국인수           0.0205      0.012      1.662      0.100      -0.004       0.045
==============================================================================
Omnibus:                       66.437   Durbin-Watson:                   1.644
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              505.195
Skew:                           2.315   Prob(JB):                    1.99e-110
Kurtosis:                      14.086   Cond. No.                     3.43e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.43e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
4914/(2245+4914)

In [ ]:
sigungu_nxy.corr(method='pearson').loc[:,'음주교통사고발생비율'].sort_values(ascending=False).head(30)

In [ ]:
sigungu_nxy.corr(method='pearson').loc['교통사고사망자수','독거노인수']

In [ ]:
sigungu_nxy.corr(method='pearson').loc['성비','음주교통사고발생비율']

In [ ]:
sigungu_nxy.corr(method='pearson').loc['외국인수','음주교통사고발생비율']

In [ ]:
##
sigungu_nxy.corr(method='pearson').loc['성비','제조업업체수']

In [ ]:
##
sigungu_nxy.corr(method='pearson').loc['외국인수','제조업업체수']

In [ ]:
sigungu_nxy.corr(method='pearson').loc['음식점및주점업종사자수','제조업업체수']

In [275]:
sigungu_nxy.corr(method='pearson').loc['음식점및주점업업체수','제조업업체수']

-0.11845536035101972

In [202]:
sigungu_nxy.corr(method='pearson').loc['음식점및주점업업체수','음주교통사고발생비율']

0.10201427428056335

In [203]:
sigungu_nxy.corr(method='pearson').loc['음식점및주점업업체수','음주교통사고발생비율']

0.10201427428056335

In [204]:
sigungu_nxy.head(1)

,년도,시도,시군구,지역,일인가구수,5대범죄 발생건수,가스사고발생건수,가해(타살) 사망자수,감염병 발생건수,감염병 사망자수,건강보험급여실적,건설업 업체수,건설업 종사자수,건축용지 면적,경찰관서수,고령인구수,공무원수(정원),공업지역 면적,교통문화지수,교통사고발생건수,교통사고부상자수,교통사고사망자수,교통사고사망자수(고속도로),구거면적,구거면적 비율,구급발생건수,구조구급 발생건수,구조구급대원수,구조발생건수,급경사지 붕괴위험지구 수,기초수급자수,기초수급자수(65세이상),노인 교통사고 부상자수,노인 교통사고 사망자수,도로면적,도로면적 비율,도로연장,도시지역 면적,도시지역 면적 비율,독거노인수,무면허교통사고 발생건수,반지하가구 수,범죄발생건수(강간.강제추행),범죄발생건수(강도),범죄발생건수(도박),범죄발생건수(마약),범죄발생건수(방화),범죄발생건수(살인),범죄발생건수(약취.유인),범죄발생건수(절도),범죄발생건수(폭력),병상수,보건업 및 사회복지서비스업 종사자수,보행사상자수,보행자전용·우선도로 면적,보행자전용·우선도로 연장,비닐하우스 면적,빈집수,산사태위험지역 면적,상업지역 면적,수계밀도,순이동자수,스트레스인지율,시가화율,시군구내 전입자수,시군구외 전입자수,시도간 전입자수,시도내-시군구간 전입자수,실업률,야간인구수,양호한 주관적 건강수준 인지율,어린이 교통사고 부상자수,어린이 교통사고 사망자수,어린이 아토피,어린이 천식,외국인수,우울감경험률,운전시 안전벨트 착용률,월간음주율,유지면적,유지면적 비율,유치원생수,음식점및주점업업체수,음식점및주점업종사자수,음주교통사고 발생건수,의료기관수(요양기관),의료보장 공.교 가입자수,의료보장 근로자 사업장수,의료보장사업장수,의료보험료,의료인력,이혼건수,익사자수,인구밀도,인플루엔자 예방접종률,임야 면적,자동차등록대수,자살 사망자수,자연재해 발생건수(10년평균),자연재해 사망자수,자연재해 사망자수(10년평균),자연재해 피해액(10년평균),자연재해 피해자수(10년평균),자연재해위험개선지구 수,장애인수,재난약자수,재정자립도,재정자주도,전기화재발생건수,제방면적,제방면적 비율,제조업업체수,제조업종사자수,주간인구지수,주거지역 면적,주민등록인구,주민등록인구(14세이하),주민등록인구(60세이상),주민등록인구(여자),주민등록인구(청소년),지역안전도(관리능력),지역안전도(등급),지역안전도(방재성능),지역안전도(위험환경),지역안전도,질병이환 및 사망외인으로 인한 사망자수,창고 및 운송관련 서비스업 업체수,초등학생수,총 사업체수,총전입자수,추락 사망자수,특수의료장비수,하천면적,하천면적 비율,하천연장,학교수(고등학교),중학교수,학교수(초등학교),해안선 길이,행정구역 면적,혼인귀화자수,화재 발생건수,화재 사망자수,화재 피해액,화재 피해자수,화재구조실적,화재사망자(환산),화재피해 경감액,fire,transport,disaster,crime,accident,suicide,infection,화재,교통,자연재해,범죄,안전사고,자살,감염병,성비,천명당외국인수,등록외국인수,전체인구,남,녀,외국인비율,group1,group2,음주교통사고발생비율
0,2015,강원도,강릉시,2015_강원도_강릉시,1161.45445,121.96083,0.04634,0.13901,25.67109,2.03886,1559476.199,37.53354,335.85565,0.02344,0.3707,1668.57424,58.84888,0.00457,77.36,228.30585,392.99003,0.83408,0.04634,0.00641,0.64111,32.34371,44.85489,2.22421,12.51118,0.41704,314.30862,108.15219,36.42143,0.32436,0.02316,2.31551,660.14293,0.13173,13.17299,404.29643,53.98342,22.52012,4.63377,0.50971,1.29746,0.55605,0.50971,0.13901,0.0,48.65458,68.02374,108.61557,309.39682,19.69352,0.0,0.0,0.0817,229.04725,99.99013,0.00176,4.9,-37.53354,23.2,2.25,773.56156,598.26604,406.61332,191.65273,2.0,9931.4202,47.9,27.61727,0.0,98.65296,89.98781,76.78157,4.7,73.9,59.8,0.00471,0.47098,70.20162,201.29097,489.5578,53.98342,14.78173,458.8359,222.69899,224.83052,6229719.75,0.41704,21.91773,0.09268,209.07,32.2,0.80524,4428.91102,3.66068,0.00231,0.0,0.0,5886.61005,0.0,0.04634,605.49472,2261.65046,17.39,64.61,2.27055,0.00123,0.12303,53.56638,300.31463,99.4,0.01616,215807.0,1283.55429,2266.60859,5034.96179,1200.47079,0.741,4.0,0.438,0.553,59.98,7.08967,4.68011,522.87461,2564.56,1371.82761,0.78774,1.52914,0.02068,2.06752,0.02235,0.50971,0.55605,1.62182,0.07086,1040.37773,9.91627,10.79668,0.09268,72985.02829,0.88042,1.57548,0.17458,271425.0696,1,0,1,1,1,1,1,4,3,4,4,4,4,4,101.4,8.25,1770,214560,106545,108015,0.824944,0.0,0.0,0.236452
